<a href="https://colab.research.google.com/github/mgrzb451/SkinLesionClassifier_v2/blob/main/slcv2_gradio_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating the Gradio Demo

In [1]:
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.9 MB/s eta 0:00:00


In [2]:
!unzip -q "skinsense_dev.zip" -d "."

# backstage.py

In [ ]:
import torch
import torchvision

# Download model specific transforms from Torchvision
transforms = torchvision.models.EfficientNet_B3_Weights.IMAGENET1K_V1.transforms()
# Load the classifier
classifier = torch.load(f="./classifier_model.pth", map_location="cpu", weights_only=False)

LESIONS_DB = {'akiec': {'name': 'Actinic keratoses or Intraepithelial Carcinoma',
  'info': '<strong>Actinic keratoses</strong><br>Rough, scaly patches on the skin caused by prolonged sun exposure.<br>Considered precancerous and can sometimes develop into skin cancer if left untreated.<br><strong>Intraepithelial (Intraepidermal) Carcinoma</strong><br>Early-stage cancer where abnormal cells are found within the epidermis without invading deeper layers.<br>Considered non-invasive, but left untreated, it can progress to invasive cancer.'},
 'bcc': {'name': 'Basal Cell Carcinoma',
  'info': 'The most common type of skin cancer.<br>Typically develops due to prolonged sun exposure and often appears as a pearly bump, a pinkish patch, or a sore that doesn’t heal.<br>It grows slowly and rarely spreads to other parts of the body, but early treatment is important.<br>It is highly treatable when detected early.'},
 'bkl': {'name': 'Seborrheic keratosis, Lichen Planus-like Keratosis or Solar Lentigo',
  'info': "Common benign skin lesions. Frequently seen adjacent to each other; they can arise from one another.<br>Typically don't require treatment. Removal is optional and usually done for cosmetic reasons."},
 'df': {'name': 'Dermatofibroma',
  'info': 'Common, benign skin growth that appears as a small, firm bump, often reddish-brown in color.<br>Harmless, rarely cause symptoms, and do not require treatment unless they become bothersome or are cosmetically concerning.<br>They may dimple inward when pressed and are usually stable in size over time.'},
 'mel': {'name': 'Melanoma',
  'info': 'A serious type of skin cancer that develops from melanocytes, the cells that produce pigment.<br>Risk factors include excessive sun exposure, fair skin, and a history of sunburns.<br>It often appears as a new or changing mole with irregular borders, uneven color, or asymmetry.<br>It can spread quickly to other parts of the body if not detected and treated early.<br>Early diagnosis and treatment are critical for the best outcomes.'},
 'nv': {'name': 'Melanocytic Nevus',
  'info': 'Commonly called moles; benign growths on the skin formed by clusters of melanocytes (pigment-producing cells).<br>Typically appear as small, round, brown or tan spots with a uniform color and smooth borders.<br>Most moles are harmless and develop during childhood or adolescence, though they can change over time.<br>Monitoring for changes in size, shape, or color is important to detect potential signs of melanoma'},
 'vasc': {'name': 'Angioma, Angiokeratoma or Pyogenic Granuloma',
  'info': 'Vascular lesions are abnormalities of the blood vessels that appear on the skin or internally.<br>They can present as red, purple, or blue patches, bumps, or discolorations, such as birthmarks, hemangiomas, or spider veins.<br>These lesions are often benign but can sometimes cause discomfort or cosmetic concerns.<br>Some vascular lesions may be associated with underlying medical conditions.'}}

LESION_LABELS = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']


def preprocess_img(img)->torch.Tensor:
  """
  Applies default transforms from Torchvision required by the model and adds a fake batch dimension.

  # Parameters:
  img - a PIL.Image returned by the image input widget

  # Returns:
  A torch.Tensor with the image uploaded by the user, preprocessed to work with the classifier

  """

  # Apply model specific transforms and add a fake batch dimension
  return torch.unsqueeze(input=transforms(img), dim=0)


def classify_img(img_tensor:torch.Tensor)->dict:
  """
  Passes an image tensor to the model to calculate classification probabilities.

  # Parameters:
  img_tensor:torch.Tensor - preprocessed image to pass to the classifier

  # Returns:
  A dictionary with 4 elements corresponding to the label and classification confidence value
  for the top 2 predictions

  """

  classifier.eval()
  with torch.inference_mode():
    # Get classification probabilities
    probabilites = torch.nn.functional.softmax(input=classifier(img_tensor), dim=1)

    # Primary classification
    first_label = LESION_LABELS[torch.argmax(probabilites)] # Symbolic label eg.: "bcc"
    first_confidence = round(probabilites.max().item()*100, 1) # Confidence of the 1st classification in %

    # Secondary classification
    second_classification = torch.sort(probabilites, descending=True)
    second_label = LESION_LABELS[second_classification.indices[0,1]]
    second_confidence = round(second_classification.values[0,1].item()*100, 1)

  return {"first_label": first_label,
          "first_confidence": first_confidence,
          "second_label": second_label,
          "second_confidence": second_confidence}

# app.py

In [5]:
import gradio as gr
from backstage import preprocess_img, classify_img, LESIONS_DB, LESION_LABELS

def run_app(user_image)->tuple[str, str]:
  """
  A wrapper function that executes the classification pipeline.

  # Parameters:
  user_image - a PIL.Image returned by the image input widget

  # Returns:
  A 2 element tuple of strings with HTML to be displayed in the classification output

  """

  # Nested function call to try to take advantage of operator fusion
  return _format_classification(classify_img(img_tensor=preprocess_img(img=user_image)))


def _format_classification(classifications:dict)->tuple[str, str]:
  """
  Takes a dictionary with the top 2 predictions and their associated certainty values and prepares HTML to be displayed
  to the user.

  # Parameters:
  classifications - a dictionary with 4 elements corresponding to the label and classification confidence value

  # Returns:
  A 2 element tuple of strings with HTML to be displayed in the classification output
  """

  first_classification = f"""<h4>The model is <span style="font-size:1.3em">{classifications["first_confidence"]}%</span> certain that it's:</h4>
<h2>{LESIONS_DB[classifications["first_label"]]["name"]}</h2>
<p><span style="font-size:1.1em">{LESIONS_DB[classifications["first_label"]]["info"]}<span style="font-size:.9em"></p>"""

  second_classification = f"""<h5>The second highest prediction with <span style="font-size:1.1em">{classifications["second_confidence"]}%</span> certainty is:</h5>
<h3>{LESIONS_DB[classifications["second_label"]]["name"]}</h3>
<p>{LESIONS_DB[classifications["second_label"]]["info"]}</p>"""

  return first_classification, second_classification


example_images = [["./examples/example1.jpg"],
                  ["./examples/example2.jpg"],
                  ["./examples/example3.jpg"]]

# The UI
with gr.Blocks(theme='CultriX/gradio-theme') as demo:

  # Title Logo
  with gr.Row(max_height=200):
    gr.Image(value="./assets/logo.png", show_label=False, show_download_button=False, container=False, interactive=False, show_fullscreen_button=False)

  # Title Text
  with gr.Row(max_height=100):
    gr.HTML("""
    <div style="text-align:center; margin:0 auto;">
      <h1>
        <span style="font-size:1.8em;">
          <span style="font-weight:bolder;">Skin</span>
          <span style="font-weight:lighter;">Sense</span>
        </span>
      </h1>
    </div>""")

  with gr.Row():
    # Instructions
    gr.HTML("""
        <h3>Tips on taking the photo</h3>
        <p>For the best results use your camera's macro mode. Make sure the area is well-lit, preferably with soft, natural light; try not to cast a shadow on the mark with the phone.</p>
        <p>Stabilize your phone to avoid blurring. I found resting the arm holding the camera near the lesion to help a lot. Some phones take better photos if you tap to manually select the focus point.</p>
        <p>Avoid digital zoom - it degrades the quality; crop the image around the spot afterward instead.</p>""")

  with gr.Row():
    # Input Column
    with gr.Column(scale=1):
      # Image upload widget
      img_input = gr.Image(label="Your Image", sources=["upload", "clipboard"], type="pil", show_fullscreen_button=False)
      # Classify button
      classify_button = gr.Button("Classify Image")

      # Example images from https://dermnetnz.org
      gr.Examples(examples=example_images, inputs=img_input)

    # Classification Output
    with gr.Column(scale=1, variant="panel"):
      first_classification_output = gr.HTML(f"""<h2>Upload a photo and click the button below the image</h2>""")
      second_classification_output = gr.HTML(f"""<h2>The classification information will appear here</h2>""")

      # Button action
      classify_button.click(fn=run_app, inputs=[img_input], outputs=[first_classification_output, second_classification_output])

  # Disclaimer and info
  gr.HTML("""
      <h2>Disclaimer</h2>
      <p>This application is not intended to be a medical diagnostic tool. The classifications are for informational purposes only and should never replace professional medical evaluation. If you have any health concerns, regarding your skin or otherwise, please seek professional medical advice.</p>


      <h2>About the Project</h2>
      <h3>Overview and Motivations</h3>
      <p>The app classifies smartphone-captured images of pigmented skin lesions into one of seven categories:</p>
      <div style="display:flex; gap:1rem;">
          <ul>
              <li>Actinic keratoses or Intraepithelial Carcinoma</li>
              <li>Angioma, Angiokeratoma or Pyogenic Granuloma</li>
              <li>Seborrheic keratosis, Solar Lentigo or<br>Lichen Planus-like Keratosis</li>
          </ul>
          <ul>
              <li>Melanoma</li>
              <li>Melanocytic Nevus</li>
              <li>Basal Cell Carcinoma</li>
              <li>Dermatofibroma</li>
          </ul>
      </div>
      <p>I wanted to leverage the amazing capabilites of AI and create an accesible and practical tool that works within real-world conditions of casual smartphone photography: uneven lighting, blurring, imperfect angles.</p>

      <h3>Technical Details</h3>
      <p>The app is built upon an <a href="https://pytorch.org/vision/main/models/generated/torchvision.models.efficientnet_b3.html" target="_blank">EfficientNet_B3</a> architecture implemented by the Pytorch team. The model was fine-tuned on the <a href="https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T" target="_blank">HAM10000 dataset.</a></p>
      <p>To address the class imbalances in the dataset the minority classes were oversampled by applying augmentations that I thought would reflect the nature of real-world photos taken with a smartphone: random rotations, horizontal/vertical flips, brightness adjustments, random blurring.</p>
      <p>The example images in the demo come from <a href="https://dermnetnz.org" target="_blank">https://dermnetnz.org</a></p>
      <p>If you want to take a look at the steps I took in developing the app check-out this <a href="https://github.com/mgrzb451/SkinLesionClassifier_v2" target="_blank">repository.</a> If you have any questions or suggestions create a new post in the <a href="link_to_discussions" target="_blank">Discussions.</a></p>


      <h2>About Me</h2>
      <p>I am very passionate about the art of coding and have been fascinated by the world of Artificial Intelligence. That said, I am not a programmer, nor do I have any formal education in computer science.</p>
      <p>This project marks my first big step in a personal journey to teach myself Python, Machine Learning, and AI from scratch. It has, at times, been really difficult but also incredibly rewarding and I am more excited than ever for future challenges!</p>
      <p>While my primary goal in developing this app was to deepen my understanding of programming and AI, I also wanted to create tangible value for others and make it as easily accessible as I could.</p>
      <p>I hope you find it useful in some way &#128578;</p>
            <h3>Contact info</h3>
            <p>I'm not really on social media, but if you want to reach out:</p>
            <ul  style="list-style: none; padding-left: 0;">
              <li style="display: flex; align-items: center; gap: 8px;"><img src="https://cdn4.iconfinder.com/data/icons/black-white-social-media/32/email_mail_envelope_send_message-256.png" alt="Email Icon" style="width:24px;height:24px;"> marcingrzyb24@o2.pl</li>
              <li style="display: flex; align-items: center; gap: 8px;"><img src="https://cdn0.iconfinder.com/data/icons/shift-logotypes/32/Github-512.png" alt="Github Logo" style="width:24px;height:24px;"><a href="https://github.com/mgrzb451" target="_blank"> My burgeoning Github page</a></li>
              <li style="display: flex; align-items: center; gap: 8px;"><img src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="Linkedin Logo" style="width:24px;height:24px;"><a href="https://www.linkedin.com/in/marcin-grzyb-421722209" target="_blank"> LinkedIn profile</a></li>
            </ul>""")

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ab0bbdb66f96b38d9a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
